# **Using HST COS data for Lyα** (Sujay)
### We only use FUVM coadded spectra

In [3]:
import altair as alt
import corv
import numpy as np
import pandas as pd

from astropy.table import Table
from astroquery.gaia import Gaia as GQ
from astroquery.simbad import Simbad as SQ
from pathlib import Path
from scripts import make_bins_alt
from tqdm import tqdm
from typing import Tuple

In [24]:
cos_objs = pd.read_csv('data/hst_cos/STAR_WD_COS_new.info', sep=r'\s,\s+', engine='python', usecols=[1, 2, 3], names=['ID', 'FLAG', 'META'])
cos_objs = cos_objs[cos_objs.META.str.contains('DA')]
cos_objs = cos_objs[~cos_objs.META.str.contains('PULSAR')].ID

gaia_ids = {}
for obj in cos_objs:
    query = SQ.query_objectids(obj).to_pandas().ID.str.decode('utf-8')
    gaia_ids[int(query[query.str.contains('DR3')].iloc[0].replace('Gaia DR3 ', ''))] = obj

params = GQ.launch_job(f'SELECT source_id, radial_velocity, teff_gspphot, logg_gspphot FROM gaiadr3.gaia_source WHERE source_id in {tuple(gaia_ids.keys())}').get_results().to_pandas()
params['NAMES'] = params.SOURCE_ID.map(gaia_ids)
params.to_csv('data/hst_cos/cos_params.csv', index=False)

In [ ]:
#make_bins_alt.read_nlte_spectrum('TYC-7218-934-1')
source_id = 'TYC 7218-934-1'
spec = Table.read(f'data/hst_cos/{source_id.replace(' ', '-')}_coadd_FUVM_final_lpALL.fits.gz').to_pandas()
params = pd.read_csv('data/hst_cos/cos_params.csv').query('SOURCE_ID == @source_id')
(params.radial_velocity.iloc[0], params.teff_gspphot.iloc[0], params.logg_gspphot.iloc[0]), spec.WAVE.values, spec.FLUX.values, 1e12*np.ones_like(spec.WAVE)

AttributeError: 'numpy.ndarray' object has no attribute 'idxmax'

In [121]:
cos_objs[137]
SQ.query_objectids('WD1547+057').to_pandas().ID.str.decode('utf-8')


0       LAMOST J154934.76+053518.2
1         SDSS J154934.75+053517.1
2           GALEX J154934.7+053518
3     Gaia DR3 4426435871273922560
4                    TIC 272675206
5                      PG 1547+057
6                      WD 1547+057
7         SDSS J154934.77+053517.1
8          USNO-B1.0 0955-00254275
9          2MASS J15493479+0535169
10        SDSS J154934.78+053517.1
11    Gaia DR2 4426435871273922560
Name: ID, dtype: object

In [16]:
tbl = Table.read('data/hst_cos/TYC-7218-934-1_coadd_FUVM_final_lpALL.fits.gz').to_pandas()
tbl.ERROR ** -2

0        inf
1        inf
2        inf
3        inf
4        inf
        ... 
58953    inf
58954    inf
58955    inf
58956    inf
58957    inf
Name: ERROR, Length: 58958, dtype: float64